<a href="https://colab.research.google.com/github/Andygrammer/DataVisualization/blob/master/ML_Step0_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Main description

The Titanic is the very first challenge of Kaggle competition.

"The competition is simple: use machine learning to create a model that predicts which passengers survived the Titanic shipwreck."

Source: https://www.kaggle.com/c/titanic

# 1. Load libraries

In [0]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# 2. Load dataset

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 3. Prepare data

In [0]:
vars = ['SEX_BINARY', 'Age']

train['Sex'].value_counts()

def convertSex(value):
  if (value == 'female'):
    return 1
  return 0

train['SEX_BINARY'] = train['Sex'].map(convertSex)
test['SEX_BINARY'] = test['Sex'].map(convertSex)

# Examples/Outputs
x = train[vars]
x = x.fillna(-1) # substitute NaN values by -1
y = train['Survived']


# 4. Create model (Random Forest with 100 decision trees)

In [9]:
# Validation set 1 (random split)
#np.random.seed(0)
#x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.5)
#print("Acc: ", np.mean(y_valid == p))

# Validation set 2 (cross validation - K-fold)
results = []
for rep in range(10):
  #print("Rep:", rep)
  k = KFold(n_splits=5, shuffle=True, random_state=rep)
  for examples, outputs in k.split(x):
    #print("Train: ", examples.shape[0])
    #print("Valid: ", outputs.shape[0])
    #print()
    x_train, x_valid = x.iloc[examples], x.iloc[outputs]
    y_train, y_valid = y.iloc[examples], y.iloc[outputs]

    # 4.1. training
    model = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
    model.fit(x_train, y_train)
    p = model.predict(x_valid)

    # 4.2. validation
    y_valid = np.mean(y_valid == p)
    results.append(y_valid)

print("Acc: ", results)
print("Acc: ", np.mean(results))

Acc:  [0.7988826815642458, 0.7359550561797753, 0.7808988764044944, 0.797752808988764, 0.7808988764044944, 0.7374301675977654, 0.7247191011235955, 0.7808988764044944, 0.7921348314606742, 0.7921348314606742, 0.7653631284916201, 0.7865168539325843, 0.7865168539325843, 0.7808988764044944, 0.7640449438202247, 0.7653631284916201, 0.7471910112359551, 0.7808988764044944, 0.7415730337078652, 0.8202247191011236, 0.7988826815642458, 0.797752808988764, 0.7752808988764045, 0.7415730337078652, 0.7471910112359551, 0.7653631284916201, 0.7415730337078652, 0.8370786516853933, 0.7471910112359551, 0.702247191011236, 0.7821229050279329, 0.8146067415730337, 0.7752808988764045, 0.7134831460674157, 0.7247191011235955, 0.7150837988826816, 0.7247191011235955, 0.8370786516853933, 0.8033707865168539, 0.7808988764044944, 0.7597765363128491, 0.7921348314606742, 0.7921348314606742, 0.7696629213483146, 0.7640449438202247, 0.7039106145251397, 0.7584269662921348, 0.7415730337078652, 0.8033707865168539, 0.79213483146067

# 5. Get results to submission

In [0]:
#sub = pd.Series(p, index=test['PassengerId'], name='Survived')
#sub.shape
#sub.to_csv("Titanic1.csv", header=True)
#!head -n10 Titanic1.csv